In [1]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import unicodecsv
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio

os.chdir('/sharedfolder/sida_classifier/')

speaker_list = ['Male', "Barack Obama", "Female"]

speaker_0_label, speaker_1_label, speaker_2_label = speaker_list

In [2]:
## Load saved features

def load_features(dir_path):
    features = []
    for filename in os.listdir(dir_path):
        with open(os.path.join(dir_path, filename)) as fi:
            csv_reader = csv.reader(fi)
            for row in csv_reader:
                features.append([float(item) for item in row])
    return features

speaker_1_features = load_features('/sharedfolder/sida_classifier/_classes_Obama_training_clips_all/Barack_Obama/_vowel_mfccs_and_deltas')
print(len(speaker_1_features))

aapb_ubm_male_features = load_features('/sharedfolder/sida_classifier/AAPB_male_vowel_mfccs_and_deltas')
print(len(aapb_ubm_male_features))

aapb_ubm_female_features = load_features('/sharedfolder/sida_classifier/AAPB_female_vowel_mfccs_and_deltas')
print(len(aapb_ubm_female_features))

17842
11531
4382


In [3]:
## Printing MFCCs and deltas for a single frame

print(random.choice(speaker_1_features))

[66.24526797595904, 47.16351117667229, 18.232871470874088, 16.889366533312195, 30.276397047380396, -3.244033673489548, -7.732952659624127, 3.4056124340447926, -1.578790188791543, 10.796785977912187, -3.475784173982231, 4.464948109583265, -11.947152493443413, -2.2069919679150196, 1.3547552962174962, -1.1988741607430005, -2.793146142827668, 3.3962972791362667, 3.5143997467029684, 1.9869421529073097, 2.2352433840041996, -0.6900260885509308, 0.4601252203730702, 2.7055261832814446, -4.64508290981342, -1.7803127876756584, -1.9924640313666004, 0.14918906558301154, -0.14290145543243166, -0.007117762637432384, 0.512618303343242, 0.8437250721685698, 0.4689839488525539, 0.21559765322202867, -0.025220439588936182, 0.0372879109118669, 0.5533389424848655, -1.0863278299424726]


In [5]:
## Combining feature sets

speaker_1_features = speaker_1_features

speaker_0_features = aapb_ubm_male_features

speaker_2_features = aapb_ubm_female_features

#print(len(speaker_1_features))
#print(len(ubm_features))

In [ ]:
## Training and multi-layer perceptron model with 9/10 of training data and evaluating performance on remaining 1/10

os.chdir('/sharedfolder/sida_classifier/')

import random
#random.shuffle(speaker_0_features)
#random.shuffle(speaker_1_features)
#random.shuffle(speaker_2_features)

from sklearn.neural_network import MLPClassifier

X = speaker_0_features[:-len(speaker_0_features)/10] + speaker_1_features[:-len(speaker_1_features)/10] + speaker_2_features[:-len(speaker_2_features)/10]
y = [1]*len(speaker_0_features[:-len(speaker_0_features)/10]) + [0]*len(speaker_1_features[:-len(speaker_1_features)/10]) + [2]*len(speaker_2_features[:-len(speaker_2_features)/10])

X_train = np.array(X)
y_train = np.array(y)

X_test = speaker_0_features[-len(speaker_0_features)/10:] + speaker_1_features[-len(speaker_1_features)/10:] + speaker_2_features[-len(speaker_2_features)/10:]
y_test = [1]*len(speaker_0_features[-len(speaker_0_features)/10:]) + [0]*len(speaker_1_features[-len(speaker_1_features)/10:]) + [2]*len(speaker_2_features[-len(speaker_2_features)/10:])

X_test = np.array(X_test)
y_test = np.array(y_test)

from sklearn.preprocessing import StandardScaler

classifier = MLPClassifier().fit(X_train_scaled, y_train)

print(classifier.score(X_test,y_test))

In [ ]:
## Training and saving an MLP model with all training data

X = speaker_1_features + ubm_features
y = [1]*len(speaker_1_features) + [0]*len(ubm_features)

classifier = MLPClassifier().fit(X, y)

trained_model_filename = speaker_1_label.replace(' ', '_') + '_vowels_mlpc_2048.pkl'
print(trained_model_filename)

## Saving trained model
joblib.dump(classifier, trained_model_filename)
classifier = joblib.load(trained_model_filename)

In [ ]:
##############################################
#### Start here to load pre-trained model ####
##############################################

trained_model_filename = speaker_1_label.replace(' ', '_') + '_vowels_mlpc_2048.pkl'

os.chdir('/sharedfolder/sida_classifier/')
classifier = joblib.load(trained_model_filename)

In [ ]:
%%capture
## Download unseen audio and split into 3-second WAV clips for testing

os.chdir('/sharedfolder/sida_classifier/')

try: os.mkdir('test_clips/')
except: pass

os.chdir('/sharedfolder/sida_classifier/test_clips/')

!wget -N http://traffic.libsyn.com/wtfpod/WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.mp3
!ffmpeg -i "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.mp3" "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.wav"
!ffmpeg -i "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA.wav" -f segment -segment_time 3 "WTF_-_EPISODE_613_PRESIDENT_BARACK_OBAMA_3_sec_%04d.wav"

In [ ]:
## Classifying short clips
#### Repeat this cell several times to help choose a classifier threshold value.

os.chdir('/sharedfolder/sida_classifier/test_clips/')

wav_pathname = os.path.abspath(random.choice([item for item in os.listdir('./') if '3_sec' in item]))

test_features = attk.get_mfccs_and_deltas(wav_pathname)

print(wav_pathname)

results = classifier.predict(test_features)  ## Predicting new observation

print(results)

vowel_results=[]

vowel_bools = attk.get_vowel_segments(wav_pathname)

print(len(results))
print(len(vowel_bools))

#for i in range(len(results)):
#    if vowel_bools[i]==True:
#        vowel_results.append(results[i])

#display(Audio(wav_pathname))

#print("All samples: "+str(np.mean(results)))
#print("Vowels only: "+str(np.mean(vowel_results)))

In [ ]:
## Classifying short clips
#### Repeat this cell several times to help choose a classifier threshold value.

os.chdir('/sharedfolder/sida_classifier/test_clips/')

wav_pathname = os.path.abspath(random.choice([item for item in os.listdir('./') if '3_sec' in item]))

test_features = attk.get_mfccs_and_deltas(wav_pathname)

print(wav_pathname)

results = classifier.predict(test_features)  ## Predicting new observation

print(results)

vowel_results=[]

vowel_bools = attk.get_vowel_segments(wav_pathname)

for i in range(len(results)):
    try:
        if vowel_bools[i]==True:
            vowel_results.append(results[i])
    except:
        pass

display(Audio(wav_pathname))

print("All samples: "+str(np.mean(results)))
print("Vowels only: "+str(np.mean(vowel_results)))

In [ ]:
## Function that classifies vowel segments only and returns 
## average output for the full clip

def classify_clip(clip_pathname):
    mfccs = attk.get_mfccs_and_deltas(clip_pathname)
    results = classifier.predict(mfccs)  ## Predicting new observation
    vowel_results=[]
    vowel_bools = attk.get_vowel_segments(clip_pathname)

    for i in range(len(results)):
        if vowel_bools[i]==True:
            vowel_results.append(results[i])

    return np.mean(vowel_results) ## Vowels only

In [ ]:
%%capture
## Classifying a long audio file

resolution_secs = 3.0

os.chdir('/sharedfolder/')

import timeit
tic=timeit.default_timer()

media_path = "/sharedfolder/sida_classifier/test_clips/20170814full.mp3"

snd = AudioFileClip.AudioFileClip(media_path)

classifications = []

for i in range(int(attk.duration(media_path)/resolution_secs)):
    try:
        snd.subclip(i * resolution_secs , (i * resolution_secs) + resolution_secs).write_audiofile('/tmp/temp_clip.wav')
        classifications.append(classify_clip('/tmp/temp_clip.wav'))
    except:
        classifications.append(0.0)
        print("Error: " + str(i))

In [ ]:
print("Time elapsed: "+str(timeit.default_timer() - tic))

In [ ]:
## Writing classification output to CSV

classifier_threshold = 0.2      # Classifier values below this threshold will be discarded

os.chdir('/sharedfolder/sida_classifier')

csv_path = media_path[:-4]+'_mlpc2048_labels_'+str(resolution_secs)+'s.csv'

counter=0

with open(csv_path,'w') as fo:
    duration = resolution_secs
    for value in classifications:
        if value >= classifier_threshold:
            start = counter * resolution_secs
            fo.write(str(start) + ',' + str(duration) +','+ str(value) + ',' + speaker_1_label + '\n')
        counter+=1